In [1]:
import pandas as pd
import numpy as np
from shapely.geometry import Point
from PIL import Image, ImageDraw
import glob
import os

In [2]:
FOLDER = "/Users/damoncrockett/Desktop/stpete/viz/tag-plots/"

In [3]:
df = pd.read_csv("./X.csv")

In [ ]:
thumb_side = 128

In [ ]:
for col in df.columns[1:10]:
    tmp = df[['local_path',col]][df[col]!=0]
    tmp.sort(col,inplace=True,ascending=False)
    tmp.reset_index(drop=True,inplace=True)
    n = len(tmp)
    
    side = int(round(np.sqrt(n))) + 5
    canvas = Image.new('RGB',(side * thumb_side, side * thumb_side),(50,50,50))
    x,y = range(side) * side, np.repeat(range(side),side)
    grid_list = pd.DataFrame(x,columns=['x'])
    grid_list['y'] = y
    
    point = []
    l = len(grid_list.index)
    for i in range(l):
        point.append(Point(grid_list.x.loc[i],grid_list.y.loc[i]))
    grid_list['point'] = point
    open_grid = list(grid_list.point)
    
    exemplar = Point(int(round(side/2)),int(round(side/2)))
    open_grid.remove(exemplar)
    
    breaks = [100,500,1000,2500]
    
    print col
    
    for i in range(n):
        im = Image.open(tmp.local_path.loc[i])
        im.thumbnail((thumb_side,thumb_side),Image.ANTIALIAS)
        draw = ImageDraw.Draw(im)
        draw.text((100,10),str(tmp[col].loc[i]),fill="white")
        
        closest_open = min(open_grid,key=lambda x: exemplar.distance(x))
        x = int(closest_open.x) * thumb_side
        y = int(closest_open.y) * thumb_side
        canvas.paste(im,(x,y))
        open_grid.remove(closest_open)
        
        if i in breaks:
            print i
        
    canvas.save(FOLDER+col+".png")